In [1]:
# Once results are available for a base year, apply observed HPMS data to 
# scale the data between a base year and a current/future year

In [6]:
import pandas as pd
df_hpms = pd.read_csv('hpms_observed.csv')

In [180]:
# Load df output
df = pd.read_csv(r'C:\Workspace\aq_tool\output_burien\2018\Burien.csv')

df['vmt'] = df['interzonal_vmt'] + df['intrazonal_vmt']
df = df.groupby(['pollutant_name','veh_type']).sum()[['total_daily_tons','vmt']].reset_index()

# Get list of all years beyond model base year
future_years = df_hpms[df_hpms.year > base_year].year.to_list()
for year in future_years:
#     
    # Get VMT scale for each year
    base_vmt = df_hpms.loc[df_hpms['year'] == base_year, county].values[0]
    future_vmt = df_hpms.loc[df_hpms['year'] == year, county].values[0]
    vmt_diff = (future_vmt-base_vmt)/base_vmt
    
    # Apply the County VMT difference between the future year and the model year to calculate local VMT impacts
    df['total_daily_tons_'+str(year)] = df['total_daily_tons'] + df['total_daily_tons']*vmt_diff
    df['vmt_'+str(year)] = df['vmt'] + df['vmt']*vmt_diff

    
# Requests often ask for data beyond HPMS availability; 
# Create extrapolation based on 2 most recent HPMS years, relative to second most recent year VMT
vmt_hpms_latest_year = df_hpms.loc[df_hpms['year'] == future_years[-1], county].values[0]
vmt_hpms_second_latest_year = df_hpms.loc[df_hpms['year'] == future_years[-2], county].values[0]
extrapolation_rate = 1+((vmt_hpms_latest_year-vmt_hpms_second_latest_year)/vmt_hpms_second_latest_year)
extrapolated_year = future_years[-1]+1

df['total_daily_tons_'+str(extrapolated_year)+'*'] = df['total_daily_tons_'+str(last_hpms_year)]*extrapolation_rate 
df['vmt_'+str(extrapolated_year)+'*'] = df['vmt_'+str(last_hpms_year)]*extrapolation_rate 

df.rename(columns={'vmt': 'vmt_'+str(base_year), 
                   'total_daily_tons': 'total_daily_tons_'+str(base_year)}, inplace=True)

# Sort columns 
year_list = [str(base_year)]+[str(i) for i in future_years]+[str(extrapolated_year)+'*']
df_vmt = df[['veh_type']+['vmt_' + i for i in year_list]].groupby('veh_type').first()
df_vmt = df_vmt.reset_index()

index_map = {'light': 1,
            'medium': 2,
            'heavy': 3,
            'transit': 4}

df_vmt['index_sort'] = df_vmt['veh_type'].map(index_map)
df_vmt.index = df_vmt['index_sort']
df_vmt = df_vmt.sort_index()
df_vmt.drop('index_sort', axis=1, inplace=True)
df_vmt.reset_index().drop('index_sort', axis=1, inplace=True)
# df_vmt.to_csv('')

In [181]:
df_vmt

,veh_type,vmt_2018,vmt_2019,vmt_2020,vmt_2021,vmt_2022*
index_sort,,,,,,
1,light,465381.419045,462727.707260,398380.783899,429941.698597,464002.963154
2,medium,15413.655993,15325.763785,13194.562795,14239.875440,15368.000872
3,heavy,2933.730777,2917.001970,2511.363623,2710.321344,2925.041090
4,transit,1727.778455,1717.926265,1479.031408,1596.204690,1722.660790


In [145]:
df

,pollutant_name,veh_type,total_daily_tons_2018,vmt_2018,total_daily_tons_2019,vmt_2019,total_daily_tons_2020,vmt_2020,total_daily_tons_2021,vmt_2021,total_daily_tons_2022*,vmt_2022*
0,Atmospheric CO2,heavy,6.249400,2933.730777,6.213764,2917.001970,5.349678,2511.363623,5.773496,2710.321344,6.230889,2925.041090
1,Atmospheric CO2,light,216.684993,465381.419045,215.449406,462727.707260,185.489008,398380.783899,200.184000,429941.698597,216.043174,464002.963154
2,Atmospheric CO2,medium,13.619913,15413.655993,13.542249,15325.763785,11.659064,13194.562795,12.582730,14239.875440,13.579571,15368.000872
3,Atmospheric CO2,transit,6.710729,1727.778455,6.672462,1717.926265,5.744590,1479.031408,6.199693,1596.204690,6.690851,1722.660790
4,CO,heavy,0.011920,2933.730777,0.011852,2917.001970,0.010204,2511.363623,0.011012,2710.321344,0.011884,2925.041090
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Total Gaseous HCs,transit,0.045290,1727.778455,0.045032,1717.926265,0.038770,1479.031408,0.041841,1596.204690,0.045156,1722.660790
84,VOCs,heavy,0.001439,2933.730777,0.001430,2917.001970,0.001232,2511.363623,0.001329,2710.321344,0.001434,2925.041090
85,VOCs,light,0.128165,465381.419045,0.127435,462727.707260,0.109714,398380.783899,0.118405,429941.698597,0.127786,464002.963154
86,VOCs,medium,0.022359,15413.655993,0.022232,15325.763785,0.019140,13194.562795,0.020657,14239.875440,0.022293,15368.000872


In [24]:
base_year_vmt = df_hpms.loc[df_hpms['year'] == base_year,'kitsap']

,pollutant_name,veh_type,total_daily_tons,vmt,vmt_2019,vmt_2020,vmt_2021,interpolated_rate,vmt_2022*
0,Atmospheric CO2,heavy,6.249400,2933.730777,2917.001970,2511.363623,2710.321344,0.079223,2925.041090
1,Atmospheric CO2,light,216.684993,465381.419045,462727.707260,398380.783899,429941.698597,0.079223,464002.963154
2,Atmospheric CO2,medium,13.619913,15413.655993,15325.763785,13194.562795,14239.875440,0.079223,15368.000872
3,Atmospheric CO2,transit,6.710729,1727.778455,1717.926265,1479.031408,1596.204690,0.079223,1722.660790
4,CO,heavy,0.011920,2933.730777,2917.001970,2511.363623,2710.321344,0.079223,2925.041090
...,...,...,...,...,...,...,...,...,...
83,Total Gaseous HCs,transit,0.045290,1727.778455,1717.926265,1479.031408,1596.204690,0.079223,1722.660790
84,VOCs,heavy,0.001439,2933.730777,2917.001970,2511.363623,2710.321344,0.079223,2925.041090
85,VOCs,light,0.128165,465381.419045,462727.707260,398380.783899,429941.698597,0.079223,464002.963154
86,VOCs,medium,0.022359,15413.655993,15325.763785,13194.562795,14239.875440,0.079223,15368.000872


In [92]:
vmt_hpms_second_latest_year

4382782.861

In [70]:
df[df['pollutant_name'] == 'CO']['vmt'].sum()

485456.5842694773

In [72]:
future_years[-1]

2021

In [75]:
df['vmt_'+str(future_years[-1])]

0       2710.321344
1     429941.698597
2      14239.875440
3       1596.204690
4       2710.321344
          ...      
83      1596.204690
84      2710.321344
85    429941.698597
86     14239.875440
87      1596.204690
Name: vmt_2021, Length: 88, dtype: float64